In [7]:
#imports
import cv2
import numpy as np
from decimal import Decimal

#getting image names
f = open("list.txt")
#setting list of images and line files
i = 0
original_imgs = []
line_files = []
line_lists = []
for line in f:
    temp = []
    original_imgs.append(cv2.imread(("05151640_0419.MP4/" + line.strip()), 1))
    #preprocessing corrct value files
    line_files.append(open("05151640_0419.MP4/" + line.strip()[0:5] + ".lines.txt"))
    l = line_files[i].readline().strip()
    while l != "":
        temp.append(l)
        l = line_files[i].readline().strip()
    if len(temp) == 3:
        temp.pop(0)
    elif len(temp) == 4:
        temp.pop(0)
        temp.pop()
    line_lists.append(temp)
    cv2.imwrite("images/img" + str(i) + ".jpg", original_imgs[i]) 
    i+=1
    
imgs = original_imgs.copy()
f.close()

FileNotFoundError: [Errno 2] No such file or directory: 'list.txt'

In [2]:
#Helpers

#region of interest function
def roi(img, vertices):
    #creating mask
    mask = np.zeros_like(img)
    #setting colour to grayscale
    colour = (128,128,128)
    #filling mask
    cv2.fillPoly(mask, vertices, colour)
    #returning cropped image
    return cv2.bitwise_and(img, mask)

#CropLayer for Neural Network
class CropLayer(object):
    def __init__(self, params, blobs):
        self.xstart = 0
        self.xend = 0
        self.ystart = 0
        self.yend = 0

    def getMemoryShapes(self, inputs):
        inputShape, targetShape = inputs[0], inputs[1]
        batchSize, numChannels = inputShape[0], inputShape[1]
        height, width = targetShape[2], targetShape[3]

        self.ystart = int((inputShape[2] - targetShape[2]) / 2)
        self.xstart = int((inputShape[3] - targetShape[3]) / 2)
        self.yend = self.ystart + height
        self.xend = self.xstart + width

        return [[batchSize, numChannels, height, width]]

    def forward(self, inputs):
        return [inputs[0][:,:,self.ystart:self.yend,self.xstart:self.xend]]

In [3]:
#Threshold image
i = 0
for img in imgs:
    (thresh, imgs[i]) = cv2.threshold(img, 150, 255, cv2.THRESH_BINARY)
    cv2.imwrite("threshold/threshold" + str(i) + ".jpg", imgs[i]) 
    i+=1

In [4]:
#canny edge detection
method = int(input("Canny(1), Neural Network(2), Laplacian(3): "))
if method == 1:
    i = 0
    for img in imgs:
        imgs[i] = cv2.Canny(img,100,150)
        cv2.imwrite("edges/edges" + str(i) + ".jpg", imgs[i]) 
        i+=1
        
#Neural Network
elif method == 2:
    #loading trained neural network
    network = cv2.dnn.readNetFromCaffe("hed-edge-detector/deploy.prototxt", "hed-edge-detector/hed_pretrained_bsds.caffemodel")
    cv2.dnn_registerLayer('Crop', CropLayer)
    i = 0
    for img in imgs:
        print("Interation: " + str(i))
        #getting binary large object from image
        blob = cv2.dnn.blobFromImage(img, scalefactor=1.0, size=(1640, 590), mean=(104.00698793, 116.66876762, 122.67891434), swapRB=False, crop=False)
        #setting input
        network.setInput(blob)
        #getting output
        output = network.forward()
        #manipulating output back to image
        output = output[0, 0]
        output = cv2.resize(output, (img.shape[1], img.shape[0]))
        output = 255 * output
        imgs[i] = output.astype(np.uint8)
        cv2.imwrite("edges/edges" + str(i) + ".jpg", imgs[i])
        i+=1

elif method == 3:
    i = 0
    for img in imgs:
        output = cv2.Laplacian(img,cv2.CV_64F)
        output = cv2.resize(output, (img.shape[1], img.shape[0]))
        output = 255 * output
        imgs[i] = output.astype(np.uint8)
        cv2.imwrite("edges/edges" + str(i) + ".jpg", imgs[i])
        i+=1

Canny(1), Neural Network(2), Laplacian(3): 1


In [5]:
#cropping region of interest
i = 0
for img in imgs:
    if len(img.shape) == 2:
        h, w = img.shape
    else:
        h, w, c = img.shape
    #Cropping to triangle shape
    verticies = [(0, h), (w / 2, h / 2), (w, h)]
    imgs[i] = roi(img, np.array([verticies], np.int32))
    cv2.imwrite("roi/roi" + str(i) + ".jpg", imgs[i])
    i+=1


In [6]:
#Hough Transform
temp_imgs = original_imgs.copy()
start_row, start_col = int(0), int(0)
end_row, end_col = int(h), int(w * .5)
i = 0
lcount = 0
rcount = 0
ltotal = 0
rtotal = 0
for img in imgs:
    if len(img.shape) == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #Splitting images
    imgleft = img[start_row:end_row , start_col:end_col]
    imgright = img[start_row:end_row , end_col:w]
    #getting left and right lane
    linesleft = cv2.HoughLines(imgleft,1,np.pi/170,10)
    linesright = cv2.HoughLines(imgright,1,np.pi/170,10)
    #writing left lane on image
    if linesleft is not None:
        for rho,theta in linesleft[0]:
            count = 0
            total = 0
            a = np.cos(theta)
            b = np.sin(theta)
            x0 = a*rho
            y0 = b*rho
            x1 = int(x0 + 1000*(-b))
            y1 = int(y0 + 1000*(a))
            x2 = int(x0 - 1000*(-b))
            y2 = int(y0 - 1000*(a))
            if abs(y1 - y2) > 300:
                cv2.line(temp_imgs[i],(x1,y1),(x2,y2),(0,0,255),2)
            if len(line_lists[i]) > 0:
                line = line_lists[i][0].split(" ")
                for j in range(0, len(line) - 1, 2):
                    if Decimal(line[j]) > x1 and Decimal(line[j]) < x2 and Decimal(line[j + 1]) < y1 and Decimal(line[j + 1]) > y2: count += 1
                    total += 1
                if count/total >= 0.5: lcount+=1
                ltotal+=1
    #writing right lane on image
    if linesright is not None:     
        for rho,theta in linesright[0]:
            count = 0
            total = 0
            a = np.cos(theta)
            b = np.sin(theta)
            x0 = a*rho
            y0 = b*rho
            x1 = int(x0 + 1000*(-b))
            y1 = int(y0 + 1000*(a))
            x2 = int(x0 - 1000*(-b))
            y2 = int(y0 - 1000*(a))
            if abs(y1 - y2) > 300:
                cv2.line(temp_imgs[i],(x1 + 820,y1),(x2 + 820,y2),(0,0,255),2)
            if len(line_lists[i]) > 1:
                line = line_lists[i][1].split(" ")
                for j in range(0, len(line) - 1, 2):
                    if Decimal(line[j]) > x1 + 820 and Decimal(line[j]) < x2 + 820 and Decimal(line[j + 1]) < y1 and Decimal(line[j + 1]) > y2: count += 1
                    total += 1
                if count/total >= 0.5: rcount+=1
                rtotal+=1
    cv2.imwrite("lines/houghlines" + str(i) + ".jpg",temp_imgs[i])
    i += 1
print("Left Accuracy: " + str(lcount/ltotal))
print("Right Accuracy: " + str(rcount/rtotal))

Left Accuracy: 0.9277777777777778
Right Accuracy: 0.0
